In [1]:
import os
from subprocess import run
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# viz libraries
import vedo as vd
import pyvista as pv
from scipy.ndimage import distance_transform_edt 

from surfrac import SurFrac
from MP_LBM_utils import write_MPLBM, check_lbm_install, read_permeability

# 1. Create a fracture surface

In [3]:
myfrac = SurFrac( h = 1.0, lx = 512, ly = 128, method = "spectral")

myfrac.params['aniso']['value'] = 0.0
myfrac.params['H']['value'] = 0.7
myfrac.params['roughness']['value'] = 4
myfrac.params['mismatch']['value'] = 0.25
myfrac.params['lambda_0']['value'] = 0.6
myfrac.params['model']['value'] = 'smooth'
myfrac.params['seed']['value'] = 1
myfrac.create_fracture()
myfrac.set_mean_aperture(15)    

--> Checking spectral Method Parameters: Starting
--> Checking spectral Method Parameters: Complete
--> Running Fracture surface method 'spectral': Starting
--> Checking spectral Method Parameters: Starting
--> Checking spectral Method Parameters: Complete


--> Generation Method: spectral
--> Method Parameter: H: {'value': 0.7, 'description': 'Hurst exponent. Determines fractal dimension. Range is (0, 1)'}
--> Method Parameter: roughness: {'value': 4, 'description': 'Root-mean-squared value of heights [m], Range is (0, infty)'}
--> Method Parameter: mismatch: {'value': 0.25, 'description': 'Mismatch length scale (wavelength) as a fraction of fracture size [0 < Mismatch < 1]'}
--> Method Parameter: N: {'value': 512, 'description': 'Discretization of fracture self.lx/self.h'}
--> Method Parameter: aniso: {'value': 0.0, 'description': 'Anisotropy Ratio [0 < Aniso < 1]. Setting to 0 is isotropic.'}
--> Method Parameter: seed: {'value': 1, 'description': 'Seed for the random number generat

/opt/anaconda3/envs/daily/lib/python3.10/site-packages/surfrac-0.2-py3.10.egg/surfrac/src/methods/spectral.py:140: RuntimeWarning: divide by zero encountered in divide
  wavelength_modulus = two_pi / q  # Wavelengths modulus


# 2. Voxelize

In [3]:
myfrac.voxelize(solid_voxels=5) 

--> Checking aperture values
--> Complete
--> Setting minimum surface value of bottom to 0
--> Complete


# 3. Visualize the 3D fracture

In [4]:
vd.settings.default_backend= 'vtk'
vdp = vd.Plotter(axes=9, bg='w', bg2='lightblue', size=(1200,900), offscreen=False)

frac_3D = myfrac.frac_3D.transpose(2,0,1)
edist = distance_transform_edt(frac_3D==0)
lego = vd.Volume(edist).legosurface(vmin=1, vmax=5).cmap('turbo',vmin=-1, vmax=5).lighting(ambient=2)
lego += vd.Volume((frac_3D==0)*1.0).legosurface(vmin=1, vmax=2).c('lightgray').opacity(0.05).lighting('shiny')

cam = dict(
            position=(452.134, 486.466, -150.875),
            focal_point=(-2.81867, 55.7670, 224.764),
            viewup=(0.782363, -0.468078, 0.410867),
            distance=730.471,
            clipping_range=(343.033, 1235.15),
            )

vdp.show( lego, camera=cam, axes=1).close()

# 4. Install LBM library

In [5]:
!git clone git@github.com:je-santos/lbm_light.git
os.chdir('lbm_light')
os.system('bash Install.sh')
os.chdir('..')

Cloning into 'lbm_light'...
X11 forwarding request failed on channel 0
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 16 (delta 3), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (16/16), 3.57 MiB | 5.18 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Archive:  palabos.zip
   creating: palabos-v2.2.1/
   creating: palabos-v2.2.1/externalLibraries/
   creating: palabos-v2.2.1/externalLibraries/tinyxml/
  inflating: palabos-v2.2.1/externalLibraries/tinyxml/tinyxml.h  
  inflating: palabos-v2.2.1/externalLibraries/tinyxml/tinyxmlparser.cpp  
  inflating: palabos-v2.2.1/externalLibraries/tinyxml/tinyxmlerror.cpp  
  inflating: palabos-v2.2.1/externalLibraries/tinyxml/readme.txt  
  inflating: palabos-v2.2.1/externalLibraries/tinyxml/tinyxml.cpp  
   creating: palabos-v2.2.1/externalLibraries/Eigen3/
  inflating: palabos-v2.2.1/externalLibraries/Eigen3/CholmodSupport  
  inflatin

  inflating: palabos-v2.2.1/externalLibraries/Eigen/src/Core/util/Memory.h  
  inflating: palabos-v2.2.1/externalLibraries/Eigen/src/Core/util/Meta.h  
  inflating: palabos-v2.2.1/externalLibraries/Eigen/src/Core/MatrixBase.h  
  inflating: palabos-v2.2.1/externalLibraries/Eigen/src/Core/Map.h  
  inflating: palabos-v2.2.1/externalLibraries/Eigen/src/Core/MathFunctions.h  
  inflating: palabos-v2.2.1/externalLibraries/Eigen/src/Core/CacheFriendlyProduct.h  
  inflating: palabos-v2.2.1/externalLibraries/Eigen/src/Core/Swap.h  
  inflating: palabos-v2.2.1/externalLibraries/Eigen/src/Core/CoreInstantiations.cpp  
  inflating: palabos-v2.2.1/externalLibraries/Eigen/src/Core/MapBase.h  
  inflating: palabos-v2.2.1/externalLibraries/Eigen/src/Core/Matrix.h  
  inflating: palabos-v2.2.1/externalLibraries/Eigen/src/Core/Flagged.h  
  inflating: palabos-v2.2.1/externalLibraries/Eigen/src/Core/GenericPacketMath.h  
  inflating: palabos-v2.2.1/externalLibraries/Eigen/src/Core/CwiseBinaryOp.h  
  

  inflating: palabos-v2.2.1/src/coProcessors/headers3D.h  
  inflating: palabos-v2.2.1/src/coProcessors/coProcessorFunctional3D.h  
  inflating: palabos-v2.2.1/src/coProcessors/coProcessorFunctional3D.hh  
  inflating: palabos-v2.2.1/src/coProcessors/coProcessor3D.h  
  inflating: palabos-v2.2.1/src/coProcessors/coProcessorInstantiation3D.hh  
  inflating: palabos-v2.2.1/src/coProcessors/coProcessor3D.hh  
   creating: palabos-v2.2.1/src/multiGrid/
  inflating: palabos-v2.2.1/src/multiGrid/headers2D.hh  
  inflating: palabos-v2.2.1/src/multiGrid/multiGridLattice2D.hh  
  inflating: palabos-v2.2.1/src/multiGrid/multiGridDataAnalysisWrapper3D.h  
  inflating: palabos-v2.2.1/src/multiGrid/interpolationHelper.h  
  inflating: palabos-v2.2.1/src/multiGrid/multiGridLattice3D.hh  
  inflating: palabos-v2.2.1/src/multiGrid/multiGridGenerator3D.h  
  inflating: palabos-v2.2.1/src/multiGrid/headers3D.hh  
  inflating: palabos-v2.2.1/src/multiGrid/multiGridOperations3D.h  
  inflating: palabos-v2

  inflating: palabos-v2.2.1/src/multiBlock/domainManipulation3D.cpp  
  inflating: palabos-v2.2.1/src/multiBlock/multiBlock2D.cpp  
  inflating: palabos-v2.2.1/src/multiBlock/sparseBlockStructure2D.cpp  
  inflating: palabos-v2.2.1/src/multiBlock/multiBlockInfo2D.cpp  
  inflating: palabos-v2.2.1/src/multiBlock/multiDataProcessorWrapper3D.h  
  inflating: palabos-v2.2.1/src/multiBlock/multiBlockOperations3D.h  
  inflating: palabos-v2.2.1/src/multiBlock/multiBlockLattice2D.h  
  inflating: palabos-v2.2.1/src/multiBlock/multiDataProcessorWrapper3D.cpp  
  inflating: palabos-v2.2.1/src/multiBlock/multiBlockSerializer2D.cpp  
  inflating: palabos-v2.2.1/src/multiBlock/reductiveMultiDataProcessorWrapper2D.cpp  
  inflating: palabos-v2.2.1/src/multiBlock/domainManipulation3D.h  
  inflating: palabos-v2.2.1/src/multiBlock/group3D.hh  
  inflating: palabos-v2.2.1/src/multiBlock/coupling3D.cpp  
  inflating: palabos-v2.2.1/src/multiBlock/serialMultiDataField3D.h  
  inflating: palabos-v2.2.1/s

-- The C compiler identification is AppleClang 14.0.0.14000029


...

-- Found MPI_C: /opt/homebrew/Cellar/open-mpi/4.1.5/lib/libmpi.dylib (found version "3.1") 
-- Found MPI_CXX: /opt/homebrew/Cellar/open-mpi/4.1.5/lib/libmpi.dylib (found version "3.1") 
-- Found MPI: TRUE (found version "3.1")  
-- Configuring done (1.6s)
-- Generating done (0.0s)
-- Build files have been written to: /Users/jesantos/Dev/tmp_fracs/SurFrac/simulations/MP_LBM/lbm_light/src/1-phase_LBM/build
[  0%] Building CXX object CMakeFiles/palabos.dir/Users/jesantos/Dev/tmp_fracs/SurFrac/simulations/MP_LBM/lbm_light/src/palabos-v2.2.1/src/algorithm/basicAlgorithms.cpp.o


Enabling POSIX


[  1%] Building CXX object CMakeFiles/palabos.dir/Users/jesantos/Dev/tmp_fracs/SurFrac/simulations/MP_LBM/lbm_light/src/palabos-v2.2.1/src/algorithm/empiricalData.cpp.o
[  2%] Building CXX object CMakeFiles/palabos.dir/Users/jesantos/Dev/tmp_fracs/SurFrac/simulations/MP_LBM/lbm_light/src/palabos-v2.2.1/src/algorithm/statistics.cpp.o
[  2%] Building CXX object CMakeFiles/palabos.dir/Users/jesantos/Dev/tmp_fracs/SurFrac/simulations/MP_LBM/lbm_light/src/palabos-v2.2.1/src/atomicBlock/atomicBlock2D.cpp.o
[  3%] Building CXX object CMakeFiles/palabos.dir/Users/jesantos/Dev/tmp_fracs/SurFrac/simulations/MP_LBM/lbm_light/src/palabos-v2.2.1/src/atomicBlock/atomicBlock3D.cpp.o
[  4%] Building CXX object CMakeFiles/palabos.dir/Users/jesantos/Dev/tmp_fracs/SurFrac/simulations/MP_LBM/lbm_light/src/palabos-v2.2.1/src/atomicBlock/atomicBlockOperations2D.cpp.o
[  5%] Building CXX object CMakeFiles/palabos.dir/Users/jesantos/Dev/tmp_fracs/SurFrac/simulations/MP_LBM/lbm_light/src/palabos-v2.2.1/src/ato

[ 36%] Building CXX object CMakeFiles/palabos.dir/Users/jesantos/Dev/tmp_fracs/SurFrac/simulations/MP_LBM/lbm_light/src/palabos-v2.2.1/src/gridRefinement/octree.cpp.o
[ 37%] Building CXX object CMakeFiles/palabos.dir/Users/jesantos/Dev/tmp_fracs/SurFrac/simulations/MP_LBM/lbm_light/src/palabos-v2.2.1/src/gridRefinement/octreeGridGenerator.cpp.o
[ 38%] Building CXX object CMakeFiles/palabos.dir/Users/jesantos/Dev/tmp_fracs/SurFrac/simulations/MP_LBM/lbm_light/src/palabos-v2.2.1/src/gridRefinement/octreeGridStructure.cpp.o
[ 38%] Building CXX object CMakeFiles/palabos.dir/Users/jesantos/Dev/tmp_fracs/SurFrac/simulations/MP_LBM/lbm_light/src/palabos-v2.2.1/src/io/colormaps.cpp.o
[ 39%] Building CXX object CMakeFiles/palabos.dir/Users/jesantos/Dev/tmp_fracs/SurFrac/simulations/MP_LBM/lbm_light/src/palabos-v2.2.1/src/io/mpiParallelIO.cpp.o
[ 40%] Building CXX object CMakeFiles/palabos.dir/Users/jesantos/Dev/tmp_fracs/SurFrac/simulations/MP_LBM/lbm_light/src/palabos-v2.2.1/src/io/multiBlockR

[ 72%] Building CXX object CMakeFiles/palabos.dir/Users/jesantos/Dev/tmp_fracs/SurFrac/simulations/MP_LBM/lbm_light/src/palabos-v2.2.1/src/multiBlock/redistribution3D.cpp.o
[ 72%] Building CXX object CMakeFiles/palabos.dir/Users/jesantos/Dev/tmp_fracs/SurFrac/simulations/MP_LBM/lbm_light/src/palabos-v2.2.1/src/multiBlock/reductiveMultiDataProcessorWrapper2D.cpp.o
[ 73%] Building CXX object CMakeFiles/palabos.dir/Users/jesantos/Dev/tmp_fracs/SurFrac/simulations/MP_LBM/lbm_light/src/palabos-v2.2.1/src/multiBlock/reductiveMultiDataProcessorWrapper3D.cpp.o
[ 74%] Building CXX object CMakeFiles/palabos.dir/Users/jesantos/Dev/tmp_fracs/SurFrac/simulations/MP_LBM/lbm_light/src/palabos-v2.2.1/src/multiBlock/serialBlockCommunicator2D.cpp.o
[ 75%] Building CXX object CMakeFiles/palabos.dir/Users/jesantos/Dev/tmp_fracs/SurFrac/simulations/MP_LBM/lbm_light/src/palabos-v2.2.1/src/multiBlock/serialBlockCommunicator3D.cpp.o
[ 75%] Building CXX object CMakeFiles/palabos.dir/Users/jesantos/Dev/tmp_frac

# 5. Check the LBM installation

In [6]:
check_lbm_install()


LBM installation was found



# 6. Create the single-phase LBM input deck

In [8]:
lbm = write_MPLBM(
                  frac_obj = myfrac, 
                  buffer_layers = 2,
                  cpus = 4,
                  num_hrs = 1,
                  allocation = None,
                  )

# 7. Run the simulation

In [9]:
os.chdir(f'{lbm.folder_path}')
os.system('bash run_frac.sh')

sh: convert: command not found
sh: convert: command not found


0

# 8. Read the permeability value

In [11]:
permeability = read_permeability()

# 9. Visualize the 3D velocity field

In [12]:
frac_size = np.array(myfrac.frac_3D.shape)
frac_size[1] = frac_size[1] + lbm.buffer_layers*2

vel_norm = pv.read('output/vtk_vel000001.vti').get_array(
                                'velocityNorm').reshape(frac_size[[2,0,1]])

vel_mesh = vel_norm[:, :, lbm.buffer_layers:-lbm.buffer_layers]
vel_thresholds = np.linspace(np.min(vel_mesh), np.max(vel_mesh), 40)
vel = vd.Volume(vel_mesh).isosurface(value=vel_thresholds)

vp = vd.Plotter(axes=9, bg='w', bg2='lightblue', size=(1200,900), offscreen=False)
vp += vel.cmap('turbo').lighting(
                    'glossy').opacity(0.6).add_scalarbar('Velocity [LBM Units]', 
                                                                  font_size=16).lighting(ambient=.25)
vp += vd.Text2D(f'The permeability of the fracture is {permeability} [l.u.]', 
                                                                        c='dr')

cam = dict(
            position=(452.134, 486.466, -150.875),
            focal_point=(-2.81867, 55.7670, 224.764),
            viewup=(0.782363, -0.468078, 0.410867),
            distance=730.471,
            clipping_range=(343.033, 1235.15),
            )
vd.show( camera=cam, axes=1).close()
